## Avalanche Gain


In [2]:
import numpy as np
def calculate_avalanche_gain(alpha: float, d: float):
  M = np.exp(alpha * d)


#### References


In [1]:
from __future__ import annotations

import math
from dataclasses import dataclass


@dataclass(frozen=True)
class TownsendResult:
    i: float                 # total current (A)
    M: float                 # avalanche gain exp(alpha*d)
    r: float                 # feedback factor gamma_e*(M-1)
    n_terms: int             # number of generations accumulated
    converged: bool          # False if breakdown/divergence or max_iter hit


def townsend_current_closed_form(i0: float, alpha: float, d: float, gamma_e: float) -> TownsendResult:
    """
    Townsend current (closed form):
        i = i0 * exp(alpha d) / (1 - gamma_e*(exp(alpha d) - 1))

    Parameters
    ----------
    i0 : float
        Primary current (A).
    alpha : float
        First Townsend coefficient (1/m).
    d : float
        Gap length (m).
    gamma_e : float
        Secondary emission coefficient (dimensionless).

    Returns
    -------
    TownsendResult
    """
    M = math.exp(alpha * d)
    r = gamma_e * (M - 1.0)

    # breakdown in this Townsend feedback model
    if r >= 1.0:
        return TownsendResult(i=float("inf"), M=M, r=r, n_terms=0, converged=False)

    i = i0 * M / (1.0 - r)
    return TownsendResult(i=i, M=M, r=r, n_terms=0, converged=True)


def townsend_current_iterative(
    i0: float,
    alpha: float,
    d: float,
    gamma_e: float,
    tol: float = 1e-12,
    max_iter: int = 10_000,
    use_tail_correction: bool = True,
) -> TownsendResult:
    """
    Iterative / generational Townsend model.

    Definitions
    -----------
    M = exp(alpha d)
    r = gamma_e*(M - 1)

    Generation 0 current: i_gen = i0*M
    Update: i_gen <- r*i_gen, accumulate i_tot += i_gen

    Convergence requires r < 1.
    """
    if i0 < 0:
        raise ValueError("i0 should be nonnegative (A).")
    if d < 0:
        raise ValueError("d should be nonnegative (m).")
    if tol <= 0:
        raise ValueError("tol must be positive.")
    if max_iter <= 0:
        raise ValueError("max_iter must be positive.")

    M = math.exp(alpha * d)
    r = gamma_e * (M - 1.0)

    if r >= 1.0:
        return TownsendResult(i=float("inf"), M=M, r=r, n_terms=0, converged=False)

    # generation 0
    i_gen = i0 * M
    i_tot = i_gen
    n_terms = 1  # counts generation 0 as a term

    # edge cases
    if i_tot == 0.0:
        return TownsendResult(i=0.0, M=M, r=r, n_terms=n_terms, converged=True)

    converged = False
    for _k in range(max_iter):
        # next generation
        i_gen *= r
        i_tot += i_gen
        n_terms += 1

        # relative contribution stopping rule
        if abs(i_gen) <= tol * abs(i_tot):
            converged = True
            break

    if (not converged) or (not use_tail_correction):
        return TownsendResult(i=i_tot, M=M, r=r, n_terms=n_terms, converged=converged)

    # analytic tail correction: remaining sum from next term onward
    # tail = i_gen*r + i_gen*r^2 + ... = i_gen*r/(1-r)
    tail = i_gen * r / (1.0 - r)
    return TownsendResult(i=i_tot + tail, M=M, r=r, n_terms=n_terms, converged=True)


def townsend_current_fixed_point(
    i0: float,
    alpha: float,
    d: float,
    gamma_e: float,
    tol: float = 1e-12,
    max_iter: int = 10_000,
) -> TownsendResult:
    """
    Fixed-point iteration for:
        i = i0*M + r*i

    i_{k+1} = i0*M + r*i_k

    Converges iff r < 1.
    """
    M = math.exp(alpha * d)
    r = gamma_e * (M - 1.0)

    if r >= 1.0:
        return TownsendResult(i=float("inf"), M=M, r=r, n_terms=0, converged=False)

    i = i0 * M  # natural initial guess
    if i == 0.0:
        return TownsendResult(i=0.0, M=M, r=r, n_terms=1, converged=True)

    converged = False
    n_terms = 0
    for _k in range(max_iter):
        i_next = i0 * M + r * i
        n_terms += 1
        if abs(i_next - i) <= tol * max(1.0, abs(i_next)):
            i = i_next
            converged = True
            break
        i = i_next

    return TownsendResult(i=i, M=M, r=r, n_terms=n_terms, converged=converged)


if __name__ == "__main__":
    # Example usage (edit numbers to your case)
    i0 = 1e-12       # A
    alpha = 200.0    # 1/m
    d = 0.01         # m
    gamma_e = 0.02   # -

    cf = townsend_current_closed_form(i0, alpha, d, gamma_e)
    it = townsend_current_iterative(i0, alpha, d, gamma_e, tol=1e-14)
    fp = townsend_current_fixed_point(i0, alpha, d, gamma_e, tol=1e-14)

    print("closed-form :", cf)
    print("iterative   :", it)
    print("fixed-point :", fp)

    # sanity check (when converged)
    if cf.converged and it.converged:
        rel_err = abs(it.i - cf.i) / max(1.0, abs(cf.i))
        print("relative error (iter vs closed):", rel_err)


closed-form : TownsendResult(i=8.471561766345384e-12, M=7.38905609893065, r=0.12778112197861302, n_terms=0, converged=True)
iterative   : TownsendResult(i=8.471561766345387e-12, M=7.38905609893065, r=0.12778112197861302, n_terms=17, converged=True)
fixed-point : TownsendResult(i=8.471273165919756e-12, M=7.38905609893065, r=0.12778112197861302, n_terms=4, converged=True)
relative error (iter vs closed): 3.2311742677852644e-27
